In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

c:\Users\VICTUS\OneDrive\Desktop\Pro\venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\VICTUS\OneDrive\Desktop\Pro\venv\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
import os
print(os.getcwd())

c:\Users\VICTUS\OneDrive\Desktop\Pro


In [3]:
model=load_model('model.h5')
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
input_data = {
    'CreditScore': 750,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 600000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\VICTUS\OneDrive\Desktop\Pro\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,France,Male,35,5,600000,2,1,1,50000


In [7]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,France,1,35,5,600000,2,1,1,50000


In [8]:
input_df=pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,1,35,5,600000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 1.02067450e+00,  9.13247552e-01, -3.70568589e-01,
        -1.34471949e-03,  8.38812313e+00,  8.08436154e-01,
         6.49202671e-01,  9.74816989e-01, -8.76832208e-01,
         1.00150113e+00, -5.79467227e-01, -5.76388018e-01]])

In [10]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


array([[0.00122633]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]

In [12]:
prediction_proba

np.float32(0.0012263301)

In [13]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
